# Outlier detection

This notebook describes a outlier/novelty detection session on Danish company data. 

It requires a comma-separated values file with features extracted from a JSONL file. 

In [1]:
# https://github.com/fnielsen/everything
from everything import *

In [2]:
# Read dataframe with features for companies
filename = expanduser('~/workspace/cvrminer/virksomheder-features.csv')
df = read_csv(filename, encoding='utf-8', index_col=0)

In [3]:
# Feature names
df.columns

Index([u'antal_penheder', u'branche_ansvarskode', u'nyeste_antal_ansatte',
       u'nyeste_virksomhedsform', u'reklamebeskyttet', u'sammensat_status',
       u'nyeste_statuskode', u'stiftelsesaar'],
      dtype='object')

In [4]:
# Functions for conversion to numerical dataframes
def to_dummies(df, column):
    datatype = df[column].dtypes
    if datatype in [int64, float64]:
        return df[[column]]
    elif datatype == bool:
        return df[[column]].astype(int)
    elif datatype == 'object':
        df_column = df[column].str.get_dummies()
        df_column.columns = [column + ":" + col for col in df_column.columns]
        return df_column
    else:
        raise ValueError('Unrecognized datatype for column {}'.format(column))
        
def dataframe_to_numerical(df):
    df_numerical = DataFrame(index=df.index)
    for column in df.columns:
        print(column)
        df_numerical = df_numerical.join(to_dummies(df, column))
    return df_numerical

In [5]:
# Numerical dataframe
dfn = dataframe_to_numerical(df)
dfn.shape

antal_penheder
branche_ansvarskode
nyeste_antal_ansatte
nyeste_virksomhedsform
reklamebeskyttet
sammensat_status
nyeste_statuskode
stiftelsesaar


(1529578, 86)

In [6]:
dfn.describe()

,antal_penheder,branche_ansvarskode:0,branche_ansvarskode:15,branche_ansvarskode:65,branche_ansvarskode:75,branche_ansvarskode:76,branche_ansvarskode:96,branche_ansvarskode:97,branche_ansvarskode:99,branche_ansvarskode:None,...,nyeste_statuskode:2,nyeste_statuskode:3,nyeste_statuskode:4,nyeste_statuskode:5,nyeste_statuskode:6,nyeste_statuskode:7,nyeste_statuskode:8,nyeste_statuskode:9,nyeste_statuskode:None,stiftelsesaar
count,1529578.000000,1529578.000000,1529578.000000,1529578.000000,1529578.000000,1529578.000000,1529578.000000,1529578.000000,1529578.000000,1529578.000000,...,1529578.000000,1529578.000000,1529578.000000,1529578.000000,1529578.000000,1529578.000000,1529578.000000,1529578.000000,1529578.000000,1525575.000000
mean,1.000951,0.001101,0.001925,0.004731,0.000669,0.000001,0.000155,0.000061,0.000110,0.991245,...,0.000142,0.024222,0.000137,0.000883,0.000114,0.000002,0.000025,0.000075,0.961358,1999.933788
std,2.352023,0.033162,0.043829,0.068622,0.025865,0.001143,0.012447,0.007839,0.010511,0.093156,...,0.011910,0.153737,0.011716,0.029706,0.010696,0.001400,0.004984,0.008671,0.192740,14.312063
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1197.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1994.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2004.000000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2010.000000
max,1323.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2016.000000


In [7]:
# Preprocessing
imputer = Imputer()
scaler = StandardScaler(with_mean=False)
dfni = scaler.fit_transform(imputer.fit_transform(dfn))

In [8]:
# Outlier detection/novelty detection with K-means clustering
clusterer = MiniBatchKMeans(n_clusters=8, random_state=1, verbose=False)
clusterer.fit(dfni)

/usr/local/lib/python2.7/dist-packages/sklearn/cluster/k_means_.py:1279: DeprecationWarning: This function is deprecated. Please call randint(0, 1529577 + 1) instead
  0, n_samples - 1, init_size)
/usr/local/lib/python2.7/dist-packages/sklearn/cluster/k_means_.py:630: DeprecationWarning: This function is deprecated. Please call randint(0, 1529577 + 1) instead
  0, n_samples - 1, init_size)
/usr/local/lib/python2.7/dist-packages/sklearn/cluster/k_means_.py:630: DeprecationWarning: This function is deprecated. Please call randint(0, 1529577 + 1) instead
  0, n_samples - 1, init_size)
/usr/local/lib/python2.7/dist-packages/sklearn/cluster/k_means_.py:630: DeprecationWarning: This function is deprecated. Please call randint(0, 1529577 + 1) instead
  0, n_samples - 1, init_size)
/usr/local/lib/python2.7/dist-packages/sklearn/cluster/k_means_.py:1328: DeprecationWarning: This function is deprecated. Please call randint(0, 1529577 + 1) instead
  0, n_samples - 1, self.batch_size)
/usr/local/l

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10, n_clusters=8,
        n_init=3, random_state=1, reassignment_ratio=0.01, tol=0.0,
        verbose=False)

In [9]:
distances = sum((dfni - clusterer.cluster_centers_[clusterer.labels_, :]) ** 2, axis=1)
indices_clusterer = argsort(-distances)

In [10]:
df.iloc[indices_clusterer[:20], :]

,antal_penheder,branche_ansvarskode,nyeste_antal_ansatte,nyeste_virksomhedsform,reklamebeskyttet,sammensat_status,nyeste_statuskode,stiftelsesaar
cvr_nummer,,,,,,,,
15706538,1,None,0,Aktieselskab,False,UNDERREASUMMATION,None,1992
20899301,1,None,2,Aktieselskab,False,UNDERREASSUMMERING,None,1998
21976415,1,None,NaN,Aktieselskab,False,OPLØSTEFTERTVANGSOPLØSNING,None,1999
36467223,1,None,NaN,Medarbejderinvesteringsselskab,False,NORMAL,None,2014
26885558,1,76,50,Fonde og andre selvejende institutioner,False,Ophørt,None,2002
35852492,1,None,NaN,Kommanditselskab,False,SLETTES,None,2014
26050286,1,76,NaN,Enkeltmandsvirksomhed,False,Ophørt,None,2001
31086477,1,None,NaN,Interessentskab,False,SLETTES,None,2007
25050193,1,None,NaN,Fast forretningssted af Europæisk økonomisk Fi...,False,Aktiv,None,1991


In [11]:
# Plot histogram for distances related to the company with the largest distance
index_for_max = indices_clusterer[0]
cluster_label = clusterer.labels_[index_for_max]
hist((distances[clusterer.labels_ == cluster_label]), bins=np.logspace(0, 8, 100))
max_value = distances[index_for_max]
ax = gca()
ax.set_xscale('log')
ax.set_yscale('symlog')
ax.annotate('Max distance', xy=(max_value, 5), xytext=(max_value, 20000), arrowprops=dict(facecolor='black'))
xlabel('Distance to cluster center for cluster label {}'.format(cluster_label))
ylabel('Absolute frequency')
title('Histogram of distances for cluster {}'.format(cluster_label))
show()

In [12]:
# Bar plot of feature distances for a company
feature_distances_for_all = (dfni - clusterer.cluster_centers_[clusterer.labels_, :]) ** 2
feature_distance = Series(feature_distances_for_all[index_for_max, :], index=dfn.columns)
feature_distance.sort_values(inplace=True, ascending=False)
feature_distance.iloc[:20][::-1].plot(kind='barh')
ax = gca()
pos = ax.get_position()
pos.x0 = 0.6
ax.set_position(pos)
ax.set_xscale('log')
xlabel('Feature distance')
title('Feature distances for most novel company')
show()

In [13]:
# Write results to an HTML file
filename = expanduser('~/workspace/cvrminer/virksomheder-report.html')
with codecs.open(filename, 'w', encoding='utf-8') as f:
    f.write("""
<html>
  <head>
    <title>Virksomheder report</title>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
  </head>
  <body>
    <h1>Virksomheder report</h1>
    
""")
    
    f.write("""<h2>Numerical features dataframe summary statistics</h2>""")
    f.write(dfn.describe().T.to_html())
    f.write("""<h2>Novelty from k-means</h2>""")
    f.write(dfn.iloc[indices_clusterer[:50], :].to_html(
            escape=False,
            formatters={'__index__': 
                        lambda idx: '<a href="http://datacvr.virk.dk/data/visenhed?enhedstype=virksomhed&id={}">{}</a>'.format(
                     idx, idx)}))
    f.write("""
  </body>
</html>""")

In [ ]:
# Investigate cluster model as a function of number of clusters
inertias = []
for n_clusters in range(1, 50):
    clusterer = MiniBatchKMeans(n_clusters=n_clusters, max_iter=200, max_no_improvement=30, n_init=10)
    clusterer.fit(dfni)
    inertias.append(clusterer.inertia_)

In [ ]:
plot(inertias)
show()